## Create test and training folders for initial CNN

In [1]:
import numpy as np
import pandas as pd

import os
from shutil import copyfile


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
#watch_df = pd.read_csv('all_watch_info.csv')
#watch_df = pd.read_csv('all_watch_info.csv')
watch_df = pd.read_csv('all_watch_info_with_indicators.csv')

In [3]:
#watch_df.groupby('image_url').count().sort_values('brand',ascending = False)

Lets start something that *seems* simple ... whether the watch is on a braclet or strap

In [4]:
X = watch_df[['new_image_name','image_name','gender']]
y = watch_df['gender']
#y = watch_df['shape_simple']

In [5]:
# 0 = womens
# 1 = mens

y = watch_df['is_male']

Uncomment this code 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

Build Directories

In [7]:
# feature = 'band_data'
# positive = 'strap'
# null = 'bracelet'

feature = 'multi_class_testing'
positive = 'male'
null = 'female'

In [8]:
def copy_train_test(feature,positive,null):

    # Build folders
    try:
        os.stat(feature)
    except:
        os.mkdir(feature) 

    # Train Data Paths
    try:
        os.stat(feature + '/train/')
    except:
        os.mkdir(feature + '/train/')   

    try:
        os.stat(feature + '/train/'+positive)
    except:
        os.mkdir(feature + '/train/'+positive)     


    try:
        os.stat(feature + '/train/'+null)
    except:
        os.mkdir(feature + '/train/'+null)      


    # Test Data Paths  
    try:
        os.stat(feature + '/test/')
    except:
        os.mkdir(feature + '/test/')   

    try:
        os.stat(feature + '/test/'+positive)
    except:
        os.mkdir(feature + '/test/'+positive)     


    try:
        os.stat(feature + '/test/'+null)
    except:
        os.mkdir(feature + '/test/'+null)      



    # Make Copies for Train Data
    for ix, val in enumerate(y_train):

        if X_train.iloc[ix,:]['gender'] == 'mens':
            f_path = 'prestige_time_pics_mens/'
        else:
            f_path = 'prestige_time_pics_womens/'

        f_name_old = X_train.iloc[ix,:]['image_name'] + '.jpg'
        f_name_new = X_train.iloc[ix,:]['new_image_name'] + '.jpg'

        if val == 1:
            f_out = feature+'/train/'+positive+'/'+f_name_new
        else:
            f_out = feature+'/train/'+null+'/'+f_name_new

        copyfile(f_path+f_name_old,f_out)


    # Make copies for test data    
    for ix, val in enumerate(y_test):

        if X_test.iloc[ix,:]['gender'] == 'mens':
            f_path = 'prestige_time_pics_mens/'
        else:
            f_path = 'prestige_time_pics_womens/'

        f_name_old = X_test.iloc[ix,:]['image_name'] + '.jpg'
        f_name_new = X_test.iloc[ix,:]['new_image_name'] + '.jpg'

        if val == 1:
            f_out = feature+'/test/'+positive+'/'+f_name_new
        else:
            f_out = feature+'/test/'+null+'/'+f_name_new

        copyfile(f_path+f_name_old,f_out)

In [9]:
copy_train_test(feature,positive,null)